## Consumer

In [1]:
import json
import pandas as pd

import time
import numpy as np
import pylab as pl

from IPython import display

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from bokeh.io import output_file, output_notebook
from itertools import chain
from bokeh.layouts import gridplot
from bokeh.io import export_png
from bokeh.models.annotations import Title

In [2]:
def create_histo(mx_name, df, figure, col):
    '''
    Function used to create the histograms of the counts
    of active TDC_CHANNEL per chamber

    Inputs:
        - mx_name:
        - df:
        - figure:
        - col:
        
    Outputs:
        - df:
        - b:
        - figure:
    '''
    
    new_bins    = mx_name[0]
    new_entries = mx_name[1]
    
    df_new = pd.DataFrame({'channel': new_bins,
                           'entries': new_entries})
    
    df = pd.merge(df, df_new, how = 'left', on = 'channel')
    df['entries'] = df[['entries_x','entries_y']].sum(axis=1)
    df = df.drop(['entries_x', 'entries_y'], axis=1)

    b = figure.vbar(x=df.channel, top=df.entries, 
                    width=0.5, alpha=0.8, color=col)
    return df, b, figure

def tdc_active_histo(mx_name, df, figure, col):
    '''
    Function used to

    Inputs:
        - mx_name:
        - df:
        - figure:
        - col:
        
    Outputs:
        - df:
        - b:
        - figure:
    '''
    
    new_bins = mx_name[0]
    new_entries = mx_name[1]
    
    df_new = pd.DataFrame({'channel': new_bins, 'entries': new_entries})
    
    df = pd.merge(df, df_new, how = 'outer', on = 'channel')
    df['entries'] = df[['entries_x','entries_y']].sum(axis=1)
    df = df.drop(['entries_x', 'entries_y'], axis=1)

    b = figure.vbar(x=df['channel'], top=df['entries'], width=0.5, alpha=0.8, color=col)
    return df, b, figure

def create_plot(handle):
    push_notebook(handle=handle)

    
def delete_histo(figure, histo):
    figure.renderers.remove(histo)

In [3]:
from kafka import KafkaConsumer

KAFKA_HOME = '/root/kafka_2.13-3.2.1'
# kafka server
KAFKA_BOOTSTRAP_SERVERS = ['localhost:9092',]

# definition of the consumer
consumer = KafkaConsumer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS, consumer_timeout_ms=20000)


consumer.topics()


{'results', 'stream', 'topic_stream'}

In [4]:
consumer.subscribe('results')
consumer.subscription()

{'results'}

In [5]:
# print the list of partition IDs 
# e.g. a topic with tree partitions will have partition IDs {0, 1, 2}
consumer.partitions_for_topic('stream')


{0}

In [6]:
'''# set up the polling strategy for the consumer
consumer.poll(timeout_ms=0,         #<<--- do not enable dead-times before one poll to the next
              max_records=None,     #<<--- do not limit the number of records to consume at once 
              update_offsets=True   #<<--- update the reading offsets on this topic
             )
'''

'# set up the polling strategy for the consumer\nconsumer.poll(timeout_ms=0,         #<<--- do not enable dead-times before one poll to the next\n              max_records=None,     #<<--- do not limit the number of records to consume at once \n              update_offsets=True   #<<--- update the reading offsets on this topic\n             )\n'

In [ ]:
output_notebook()

my_figure0 = figure(title='Hits in Chamber 0')
my_figure1 = figure(title='Hits in Chamber 1')
my_figure2 = figure(title='Hits in Chamber 1')
my_figure3 = figure(title='Hits in Chamber 1')

tdc_fig0 = figure(title='Active TDC channels per orbit in Chamber 0')
tdc_fig1 = figure(title='Active TDC channels per orbit in Chamber 1')
tdc_fig2 = figure(title='Active TDC channels per orbit in Chamber 2')
tdc_fig3 = figure(title='Active TDC channels per orbit in Chamber 3')

scint_fig0 = figure()
scint_fig1 = figure()
scint_fig2 = figure()
scint_fig3 = figure()

bins = []
entries = []


test_data = ColumnDataSource(data=dict(y=entries, x=bins))

handle = show(gridplot([my_figure0, my_figure1, my_figure2, my_figure3,
                       tdc_fig0, tdc_fig1, tdc_fig2, tdc_fig3,
                       scint_fig0, scint_fig1, scint_fig2, scint_fig3], 
                       ncols=2, width=200, height=200, 
                      toolbar_location=None), notebook_handle=True)

plot = gridplot([my_figure0, my_figure1, my_figure2, my_figure3,
                       tdc_fig0, tdc_fig1, tdc_fig2, tdc_fig3], 
                       ncols=2, width=300, height=300, 
                      toolbar_location=None)

push_notebook(handle=handle)

    


bins_ch0 = np.arange(0, 64, 1)
bins_ch1 = np.arange(64, 128, 1)
count = np.zeros_like(bins_ch0)

df_ch0 = pd.DataFrame({'channel':bins_ch0, 'entries':count})
df_ch1 = pd.DataFrame({'channel':bins_ch1, 'entries':count})
df_ch2 = pd.DataFrame({'channel':bins_ch0, 'entries':count})
df_ch3 = pd.DataFrame({'channel':bins_ch1, 'entries':count})
df_list = [df_ch0, df_ch1, df_ch2, df_ch3]




active_channels = np.arange(0,2,1)
counts_channels = np.zeros_like(active_channels)

tdc_0 = pd.DataFrame({'channel':active_channels, 'entries':counts_channels})
tdc_1 = pd.DataFrame({'channel':active_channels, 'entries':counts_channels})
tdc_2 = pd.DataFrame({'channel':active_channels, 'entries':counts_channels})
tdc_3 = pd.DataFrame({'channel':active_channels, 'entries':counts_channels})
tdc_list = [tdc_0, tdc_1, tdc_2, tdc_3]




scint_0 = pd.DataFrame({'channel':bins_ch0, 'entries':count})
scint_1 = pd.DataFrame({'channel':bins_ch1, 'entries':count})
scint_2 = pd.DataFrame({'channel':bins_ch0, 'entries':count})
scint_3 = pd.DataFrame({'channel':bins_ch1, 'entries':count})
scint_list = [scint_0, scint_1, scint_2, scint_3]

images = []


index = 0
for message in consumer:
        
    mx = json.loads(message.value)

    print('Number of hits: \t', mx['hits'], 
          '\n Number of hits post-cleansing in each chamber: \t %s' % 
          str(mx['hitsPerChamber'])
         )

    name_list = [mx['histo_ch0'], mx['histo_ch1'], mx['histo_ch2'], 
                 mx['histo_ch3']]

    df_list[0], a, my_figure0= create_histo(name_list[0], df_list[0], 
                                            my_figure0, col='blue')

    df_list[1], b, my_figure1= create_histo(name_list[1], df_list[1], 
                                            my_figure1, col='red')

    df_list[2], c, my_figure2= create_histo(name_list[2], df_list[2], 
                                            my_figure2, col='yellow')

    df_list[3], d, my_figure3= create_histo(name_list[3], df_list[3], 
                                            my_figure3, col='green')





    name_tdc = [mx['histo_orbit_ch0'], 
                mx['histo_orbit_ch1'], 
                mx['histo_orbit_ch2'], 
                mx['histo_orbit_ch3']]

    tdc_list[0], e, tdc_fig0 = tdc_active_histo(name_tdc[0], tdc_list[0], tdc_fig0, col='blue')
    tdc_list[1], f, tdc_fig1 = tdc_active_histo(name_tdc[1], tdc_list[1], tdc_fig1, col='red')
    tdc_list[2], g, tdc_fig2 = tdc_active_histo(name_tdc[2], tdc_list[2], tdc_fig2, col='yellow')
    tdc_list[3], h, tdc_fig3 = tdc_active_histo(name_tdc[3], tdc_list[3], tdc_fig3, col='green')

    scint_names = [mx['histo_scin_ch0'],
                  mx['histo_scin_ch1'],
                  mx['histo_scin_ch2'],
                  mx['histo_scin_ch3']]

    scint_list[0], i, scint_fig0 = create_histo(scint_names[0], scint_list[0], scint_fig0, col='blue')
    scint_list[1], j, scint_fig1 = create_histo(scint_names[1], scint_list[1], scint_fig1, col='red')
    scint_list[2], k, scint_fig2 = create_histo(scint_names[2], scint_list[2], scint_fig2, col='yellow')
    scint_list[3], l, scint_fig3 = create_histo(scint_names[3], scint_list[3], scint_fig3, col='green')


    create_plot(handle)

    delete_histo(my_figure0, a)
    delete_histo(my_figure1, b)
    delete_histo(my_figure2, c)
    delete_histo(my_figure3, d)
    delete_histo(tdc_fig0, e)
    delete_histo(tdc_fig1, f)
    delete_histo(tdc_fig2, g)
    delete_histo(tdc_fig3, h)
    delete_histo(scint_fig0, i)
    delete_histo(scint_fig1, j)
    delete_histo(scint_fig2, k)
    delete_histo(scint_fig3, l)


    images.append(plot)
    #print(type(plot))
    index += 1
    if index > 25:
        break


Loading BokehJS ...